In [ ]:
import cv2 as cv 
import matplotlib.pyplot as plt 
import numpy as np 
from collections import defaultdict

In [ ]:
image_path = 'Resources_Coins01.jpeg'

# Load the image
image = cv.imread(image_path, cv.IMREAD_GRAYSCALE)

# Apply Gaussian blur to reduce noise
blurred = cv.GaussianBlur(image, (5, 5), 0)

# Canny edge detection
edges = cv.Canny(blurred, 50, 180)

# Display the original image and the edges
cv.imshow('Original Image', image)
cv.imshow('Canny Edges', edges)
cv.waitKey(0)
cv.destroyAllWindows()

# Circle

In [ ]:
#dp to control accumulator size, min_dist to control accepted circles according to the distance bet. their centers
def hough_circle_transform(edge_image, min_radius, max_radius, accumulator_threshold, min_dist):
    # Define Hough space dimensions based on edge_image size and radius range
    height, width = edge_image.shape[:2]

    # Define radius range for iteration
    radii = np.arange(min_radius, max_radius + 1)

    circle_candidates = []
    for radius in radii:
        for theta in np.linspace(0, 2*np.pi, 100):
            circle_candidates.append((radius,int(radius * np.cos(theta)),int(radius * np.sin(theta))))

    # Hough Accumulator, we are using defaultdic instead of standard dict as this will initialize for key which is not 
    # aready present in the dictionary instead of throwing exception.
    accumulator = defaultdict(int)
    
    # Find edge pixels in the image to get (x,y) in image space
    edge_pixels = np.argwhere(image > 0)
    for y in range(height):
        for x in range(width):
            if edge_image[y][x] != 0: #white pixel (edge)
                for r, r_cos, r_sin in circle_candidates:
                    a = x - r_cos
                    b = y - r_sin
                    accumulator[(a, b, r)] += 1 #vote for current candidate

    # Output image with detected lines drawn
    output_img = image.copy()
    # Output list of detected circles. A single circle would be a tuple of (x,y,r,threshold) 
    out_circles = []

    # Sort the accumulator based on the votes for the candidate circles 
    for candidate_circle, votes in sorted(accumulator.items(), key=lambda i: -i[1]):
        x, y, r = candidate_circle
        current_vote = votes
        if current_vote >= accumulator_threshold: 
            # Shortlist the circle for final result
            out_circles.append((x, y, r, current_vote))
            print(x, y, r, current_vote)

    filtered_circles = []
    for x, y, r, v in out_circles:
    # Exclude circles that are too close of each other
    # all((x - xc) ** 2 + (y - yc) ** 2 > rc ** 2 for xc, yc, rc, v in filtered_circles)
    # Remove nearby duplicate circles based on min_dist
        if all(abs(x - xc) > min_dist or abs(y - yc) > min_dist or abs(r - rc) > min_dist for xc, yc, rc, v in filtered_circles):
            filtered_circles.append((x, y, r, v))
    out_circles = filtered_circles

    # Draw shortlisted circles on the output image
    for x, y, r, v in out_circles:
        output_img = cv.circle(output_img, (x,y), r, (0,255,0), 3)
    
    return output_img




min_radius = 10
max_radius = 80
accumulator_threshold = 52
min_dist = 30  # Adjust this value as needed

circle_img = hough_circle_transform(edges, min_radius, max_radius, accumulator_threshold, min_dist)

# Draw detected circles on the grayscale version of the original image
cv.imshow('Detected Circles', circle_img)
cv.waitKey(0)
cv.destroyAllWindows()


In [ ]:

# Load the image
image = cv.imread('Resources_Coins01.jpeg', cv.IMREAD_GRAYSCALE)

# Apply Gaussian blur to reduce noise
blurred_image = cv.GaussianBlur(image, (5, 5), 0)

# Detect circles using Hough Circle Transform
circles = cv.HoughCircles(edges, cv.HOUGH_GRADIENT, dp=1, minDist=20,
                           param1=50, param2=30, minRadius=10, maxRadius=50)

# If circles are detected, draw them
if circles is not None:
    circles = np.round(circles[0, :]).astype("int")
    for (x, y, r) in circles:
        cv.circle(image, (x, y), r, (0, 255, 0), 3)

# Display the image with detected circles
cv.imshow("Detected Circles (OpenCV)", image)
cv.waitKey(0)
cv.destroyAllWindows()
